In [1]:
from pymongo import MongoClient
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import MongoDBAtlasVectorSearch
from langchain_community.document_loaders import DirectoryLoader
from langchain_openai import OpenAI
from langchain.chains import RetrievalQA
import gradio as gr
from gradio.themes.base import Base
import key_param

/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/timhulshof/Documents/RAG/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MongoClient(key_param.MONGO_URI)
dbName = "PKU"
collectionName = "collection_of_pku_text_blobs"
collection = client[dbName][collectionName]

In [3]:
loader = DirectoryLoader('./sample_files/', glob='./*.txt', show_progress=True)
data = loader.load()

embeddings = OpenAIEmbeddings(openai_api_key=key_param.openai_api_key)

  0%|          | 0/14 [00:00<?, ?it/s]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/timhulshof/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
  7%|▋         | 1/14 [00:02<00:28,  2.23s/it][nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/timhulshof/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 14/14 [00:02<00:00,  4.94it/s]


In [4]:
vectorStore = MongoDBAtlasVectorSearch.from_documents(data, embeddings, collection=collection)